In [1]:
import numpy as np
import torch
from matplotlib import pyplot as plt
from sklearn.linear_model import LinearRegression
from torch import nn

from data import get_loaders, load_dataset
from training import test_step, train_step, train_model, get_test_predictions
from utils import print_losses
from swag import sample_from_SWAG, run_SWAG
from definitions import DATASETS
from models import create_model, load_model
from metrics import RMSE
from swag_mod import calculate_coeffs

In [2]:
def train_step_mod(
    batch_x: torch.Tensor,
    batch_y: torch.Tensor,
    model: nn.Module,
    optimizer: torch.optim.Optimizer,
    criterion,
    lr_multipliers,
):
    optimizer.zero_grad()
    outputs = model(batch_x)
    loss = criterion(outputs, batch_y)
    loss.backward()
    multitiply_grads(model, lr_multipliers)
    optimizer.step()
    return loss.item()

def train_SWAG_mod(
    x_train: np.array,
    y_train: np.array,
    x_test: np.array,
    y_test: np.array,
    model: nn.Module,
    K,
    lr_multipliers,
    epochs: int = 100,
    batch_size: int = 100,
    lr: float = 0.1,
    verbose: bool = True,
    c=1,
    momentum=0,
    weight_decay=0,
):
    assert c >= 1 and K >= 2
    train_loader, test_loader = get_loaders(x_train, y_train, x_test, y_test, batch_size)
    theta_epoch = torch.nn.utils.parameters_to_vector(model.parameters()).detach().cpu().clone()
    theta = theta_epoch.clone()
    theta_square = theta_epoch.clone() ** 2
    D = None

    optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=momentum, weight_decay=weight_decay)
    criterion = nn.MSELoss()

    train_losses = [test_step(batch_x, batch_y, model, criterion) for batch_x, batch_y in train_loader]
    test_losses = [test_step(batch_x, batch_y, model, criterion) for batch_x, batch_y in test_loader]
    if verbose:
        print_losses(0, train_losses, test_losses)

    thetas = []
    for epoch in range(1, epochs + 1):
        train_losses = [train_step_mod(batch_x, batch_y, model, optimizer, criterion, lr_multipliers) 
                        for batch_x, batch_y in train_loader]
        test_losses = [test_step(batch_x, batch_y, model, criterion) for batch_x, batch_y in test_loader]
        if verbose:
            print_losses(epoch, train_losses, test_losses)
        if epoch % c == 0:
            if verbose:
                print("SWAG moment update")
            n = epoch / c
            theta_epoch = torch.nn.utils.parameters_to_vector(model.parameters()).detach().cpu().clone()
            thetas.append(theta_epoch.clone())
            theta = (n * theta + theta_epoch.clone()) / (n + 1)
            theta_square = (n * theta_square + theta_epoch.clone() ** 2) / (n + 1)
            deviations = (theta_epoch.clone() - theta).reshape(-1, 1)
            if D is None:
                D = deviations
            else:
                if D.shape[1] == K:
                    D = D[:, 1:]
                D = torch.cat((D, deviations), dim=1)
    sigma_diag = theta_square - theta ** 2
    torch.nn.utils.vector_to_parameters(theta, model.parameters())
    test_losses = [test_step(batch_x, batch_y, model, criterion) for batch_x, batch_y in test_loader]
    # print(f"Finished SWAG.     Best test loss: {np.mean(test_losses):.5f}")
    return theta, sigma_diag, D, thetas

def multitiply_grads(model, lr_multipliers):
    start_ind = 0
    for params in model.parameters():
        shape = params.shape
        total_len = params.reshape(-1).shape[0]
        multipliers = lr_multipliers[start_ind:(start_ind + total_len)].reshape(shape)
        start_ind += total_len
        params.grad = params.grad * multipliers

In [3]:
# SWAG LOOP

# for dataset_name in DATASETS[:3]:
#     run_SWAG(dataset_name, weight_decay=0)

In [ ]:
K = 10
S = 500
weight_decay = 0
multiplier = 2
tolerance = 0.05
# for dataset_name in ['boston_housing']:
# for dataset_name in ['yacht']:
# for dataset_name in ['year_prediction_msd']:
# for dataset_name in ['naval_compressor_decay']:
for dataset_name in DATASETS[:5]:
    run_SWAG(dataset_name, weight_decay=0)
    x_train, y_train, x_test, y_test, _, _ = load_dataset(dataset_name, verbose=False)
    batch_size = x_train.shape[0]//9
    model = create_model(x_train, layer_dims=[50], verbose=False)
#     train_model(x_train, y_train, x_test, y_test, model, dataset_name, lr=0.001, epochs=50000, verbose=False, 
#                 batch_size=batch_size, weight_decay=weight_decay)
    model = load_model(model, f"best_model_weights-{dataset_name}.pth", verbose=False)
    y_pred = get_test_predictions(x_train, y_train, x_test, y_test, model)
#     print(f"SGD RMSE: {RMSE(y_pred, y_test):.3f}")
    print("---MOD---")
    theta_epoch = torch.nn.utils.parameters_to_vector(model.parameters()).detach().cpu().clone()
    lr_multipliers = torch.ones_like(theta_epoch).float()
    for rounds in range(12):
        try:
#             print(0.001 * (2 ** (rounds)))
            model = load_model(model, f"best_model_weights-{dataset_name}.pth", verbose=False)
            theta_swa, sigma_diag, D, thetas = train_SWAG_mod(x_train, y_train, x_test, y_test, model, K, lr_multipliers,
                                                          verbose=False, lr=0.001, batch_size=batch_size, 
                                                          weight_decay=weight_decay, epochs=50)
            weight_series = torch.stack(thetas).numpy()
            step_plot = weight_series.shape[1] // 5
            weight_series -= weight_series.mean(axis=0, keepdims=True)
            weight_series /= weight_series.std(axis=0, keepdims=True)
#             plt.plot(weight_series[:,::step_plot], alpha=0.3)
#             plt.show()
            coeffs = calculate_coeffs(weight_series, False)
#             print(coeffs)
            lr_multipliers[np.abs(coeffs) > tolerance] *= multiplier
#             print(lr_multipliers)
            sigma_diag = torch.clamp(sigma_diag, min=1e-10)
            samples = sample_from_SWAG(x_train, y_train, x_test, y_test, model, theta_swa, sigma_diag, D, K, S)
            samples_array = np.concatenate(samples, axis=1)
            y_pred = samples_array.mean(axis=1, keepdims=True)
            y_l = np.percentile(samples_array, 2.5, axis=1, keepdims=True)
            y_u = np.percentile(samples_array, 97.5, axis=1, keepdims=True)
            print(f"RMSE: {RMSE(y_pred, y_test):.3f}, PICP: {np.mean((y_l < y_test) & (y_test < y_u)):.3f}, MPIW:{np.mean(y_u - y_l):.3f}")
        except Exception as e:
#             raise e
            print("Some error")
    

dataset: boston_housing, rows: 506, columns: 13, range of x: [0.0, 711.0], range of y: [5.0, 50.0]
Finished Training. Best validation loss: 0.13997 in epoch 258
SGD RMSE: 0.346
SWAG_lr:  0.01000
RMSE: 0.367, PICP: 0.255, MPIW:0.196
---MOD---
RMSE: 0.349, PICP: 0.059, MPIW:0.028
RMSE: 0.350, PICP: 0.098, MPIW:0.069
RMSE: 0.353, PICP: 0.078, MPIW:0.084
RMSE: 0.363, PICP: 0.235, MPIW:0.138
RMSE: 0.379, PICP: 0.275, MPIW:0.244
RMSE: 0.396, PICP: 0.412, MPIW:0.373
RMSE: 0.415, PICP: 0.451, MPIW:0.428
RMSE: 0.429, PICP: 0.451, MPIW:0.443
RMSE: 0.425, PICP: 0.510, MPIW:0.526
RMSE: 0.449, PICP: 0.529, MPIW:0.597
RMSE: 0.456, PICP: 0.549, MPIW:0.609
RMSE: 0.437, PICP: 0.627, MPIW:0.704
dataset: concrete, rows: 1030, columns: 8, range of x: [0.0, 1145.0], range of y: [2.33, 82.6]
Finished Training. Best validation loss: 0.35406 in epoch 241
SGD RMSE: 0.567
SWAG_lr:  0.20000
RMSE: 0.355, PICP: 0.777, MPIW:0.928
---MOD---
RMSE: 0.560, PICP: 0.019, MPIW:0.037
RMSE: 0.553, PICP: 0.058, MPIW:0.069
RM